In [1]:

import os
os.environ['CUDA_VISIBLE_DEVICES']='0'
import copy
import logging
import numpy as np
import pandas as pd
from datetime import datetime,timedelta,date
import time
import torch
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader,RandomSampler
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModel, AutoModelForSeq2SeqLM, AutoModel, AutoModelForCausalLM
from sentence_transformers import SentencesDataset, LoggingHandler, SentenceTransformer
from sklearn.metrics.pairwise import paired_cosine_distances, paired_euclidean_distances, paired_manhattan_distances
from scipy.stats import pearsonr, spearmanr
from transformers import BertConfig, BertTokenizer,BertTokenizerFast
from torch.nn.utils.rnn import pad_sequence
from torch.types import Number
from typing import List,Union,Dict
import tqdm
import re
import json
from dataclasses import dataclass
from pprint import pprint
os.chdir('/home/stops/Work_space/NLP_work/Med_assit_chatglm')

logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO,format='%(asctime)s - %(levelname)s - %(message)s')
logger.info('Starting')


def show_df(df):
    print(df.shape)
    print(df.head(2))
    print(df.isnull().sum())
    


2024-01-25 10:28:15,542 - INFO - Starting


In [2]:
data_file="data/问模板-话术标准模版-标注参考-20231013.xlsx"

data_df=pd.read_excel(data_file)
show_df(data_df)



(1750, 5)
   id 模板名称  标签名称  标签code                                              内容-清洗
0  26  糖尿病  主要症状       1  您发现血糖升高有多长时间了？平时有无多饮、多食、多尿、体重下降等情况呢？\n最高血糖数值是多...
1  26  糖尿病  伴随症状       2                     平时有自觉视物模糊、四肢麻木、皮肤瘙痒、或小便量少伴异味吗？
id        0
模板名称      0
标签名称      0
标签code    0
内容-清洗     0
dtype: int64


In [3]:

print(data_df["标签名称"].unique())


['主要症状' '伴随症状' '病因诱因' '诊疗经过' '既往史']


In [4]:

select_df=data_df.loc[:,["内容-清洗","标签名称"]].copy()
select_df.columns=["text","label_answer"]

show_df(select_df)


(1750, 2)
                                                text label_answer
0  您发现血糖升高有多长时间了？平时有无多饮、多食、多尿、体重下降等情况呢？\n最高血糖数值是多...         主要症状
1                     平时有自觉视物模糊、四肢麻木、皮肤瘙痒、或小便量少伴异味吗？         伴随症状
text            0
label_answer    0
dtype: int64


In [5]:

select_df.loc[:,"text_len"]=select_df.loc[:,"text"].map(len)
print(select_df["text_len"].describe(percentiles=[0.01,0.1,0.2,0.25,0.5,0.75,0.8,0.9,0.99,0.9999]))


count     1750.000000
mean        50.116000
std         27.101945
min          1.000000
1%          14.000000
10%         24.000000
20%         29.000000
25%         31.000000
50%         43.000000
75%         61.000000
80%         68.000000
90%         88.000000
99%        134.000000
99.99%     211.327100
max        215.000000
Name: text_len, dtype: float64


In [6]:
label_data=data_df["标签名称"].unique().tolist()
label_data.append("其他")
logger.info(f"label_data: {label_data}")

label2id_data=dict(zip(label_data,range(len(label_data))))
logger.info(f"label2id_data: {label2id_data}")


2024-01-25 11:13:03,597 - INFO - label_data: ['主要症状', '伴随症状', '病因诱因', '诊疗经过', '既往史', '其他']
2024-01-25 11:13:03,598 - INFO - label2id_data: {'主要症状': 0, '伴随症状': 1, '病因诱因': 2, '诊疗经过': 3, '既往史': 4, '其他': 5}


In [7]:


save_mode=True
save_file="data/doc_type_label2id_240125.json"
if save_mode:
    with open(save_file,"w+") as f :
        json.dump(label2id_data,f,indent=2,ensure_ascii=False)
    logger.info(f"save file : {save_file}")


In [8]:

save_mode=True
if save_mode:
    save_file="output_data/Doctor_query_type_clf_train_data_v0_240125.xlsx"
    select_df.to_excel(save_file,index=False)
    logger.info(f'data shape: {select_df.shape}, save file : {save_file}')



2024-01-25 11:15:30,412 - INFO - data shape: (1750, 3), save file : output_data/Doctor_query_type_clf_train_data_v0_240125.xlsx
